# Experiment 6: Noise Evolution & Position Analysis

**Key Finding:** α decreases with token position due to:
1. LayerNorm washing out noise
2. Attention dilution over longer context

**Implication:** Dynamic T*(t) might be better than constant T*

In [ ]:
# ============================================================
# UTILITIES - Run this cell first!
# ============================================================

"""
Utility functions for quantization noise and temperature experiments.
"""

import torch
import torch.nn.functional as F
import numpy as np
from contextlib import contextmanager
import re


# =============================================================================
# Noise Injection
# =============================================================================

@contextmanager
def weight_noise_context(model, noise_scale):
    """
    Context manager that adds Gaussian noise to model weights.
    Noise is removed when exiting the context.
    """
    original_weights = {}
    
    try:
        # Add noise to weights
        for name, param in model.named_parameters():
            if 'weight' in name and param.requires_grad is False:
                original_weights[name] = param.data.clone()
                noise = torch.randn_like(param.data) * noise_scale * param.data.std()
                param.data.add_(noise)
        
        yield model, original_weights
        
    finally:
        # Restore original weights
        for name, param in model.named_parameters():
            if name in original_weights:
                param.data.copy_(original_weights[name])


@contextmanager  
def activation_noise_context(model, noise_scale):
    """
    Context manager that adds Gaussian noise to activations via forward pre-hooks.
    Uses INPUT hooks (pre-hook) for realistic quantization simulation.
    """
    handles = []
    
    def make_hook(scale):
        def hook(module, args, kwargs):
            if len(args) > 0:
                x = args[0]
                noise = torch.randn_like(x) * scale * x.std()
                noisy_x = x + noise
                return (noisy_x,) + args[1:], kwargs
            return args, kwargs
        return hook
    
    try:
        # Register pre-hooks on linear layers
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                handle = module.register_forward_pre_hook(make_hook(noise_scale), with_kwargs=True)
                handles.append(handle)
        
        yield model, handles
        
    finally:
        # Remove hooks
        for handle in handles:
            handle.remove()


@contextmanager
def combined_noise_context(model, weight_noise_scale, activation_noise_scale):
    """
    Context manager that adds both weight and activation noise.
    """
    handles = []
    original_weights = {}
    
    def make_hook(scale):
        def hook(module, args, kwargs):
            if len(args) > 0:
                x = args[0]
                noise = torch.randn_like(x) * scale * x.std()
                noisy_x = x + noise
                return (noisy_x,) + args[1:], kwargs
            return args, kwargs
        return hook
    
    try:
        # Add weight noise
        for name, param in model.named_parameters():
            if 'weight' in name and param.requires_grad is False:
                original_weights[name] = param.data.clone()
                noise = torch.randn_like(param.data) * weight_noise_scale * param.data.std()
                param.data.add_(noise)
        
        # Add activation hooks
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                handle = module.register_forward_pre_hook(make_hook(activation_noise_scale), with_kwargs=True)
                handles.append(handle)
        
        yield model, (original_weights, handles)
        
    finally:
        # Restore weights
        for name, param in model.named_parameters():
            if name in original_weights:
                param.data.copy_(original_weights[name])
        
        # Remove hooks
        for handle in handles:
            handle.remove()


# =============================================================================
# Alpha (noise-to-signal ratio) Measurement
# =============================================================================

def compute_alpha(clean_logits, noisy_logits):
    """
    Compute noise-to-signal ratio α = σ²_noise / τ²_signal
    
    Args:
        clean_logits: Logits from clean model
        noisy_logits: Logits from noisy model
    
    Returns:
        dict with tau_sq, sigma_sq, alpha, t_star
    """
    diff = noisy_logits - clean_logits
    tau_sq = clean_logits.var().item()
    sigma_sq = diff.var().item()
    alpha = sigma_sq / tau_sq if tau_sq > 0 else 0
    t_star = np.sqrt(1 + alpha)
    
    return {
        'tau_sq': tau_sq,
        'sigma_sq': sigma_sq,
        'alpha': alpha,
        't_star': t_star
    }


def get_logits(model, tokenizer, prompt):
    """Get logits for next token prediction."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(inputs.input_ids)
    return outputs.logits[0, -1, :].float()


# =============================================================================
# Generation with Temperature (FIXED - uses sampling!)
# =============================================================================

def generate_with_temperature(model, tokenizer, prompt, temperature=1.0, 
                              max_new_tokens=512, do_sample=True, seed=None):
    """
    Generate text with proper temperature scaling.
    
    IMPORTANT: Temperature only affects sampling, not argmax!
    - do_sample=True: temperature scales probabilities before sampling
    - do_sample=False: temperature has NO effect (argmax is scale-invariant)
    
    Args:
        model: The language model
        tokenizer: The tokenizer
        prompt: Input prompt string
        temperature: Sampling temperature (only works with do_sample=True)
        max_new_tokens: Maximum tokens to generate
        do_sample: If True, sample from distribution. If False, greedy (T ignored!)
        seed: Random seed for reproducibility
    
    Returns:
        generated_text: The generated string
        generated_tokens: List of token ids
    """
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
    
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_ids = inputs.input_ids.clone()
    generated_tokens = []
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids)
            logits = outputs.logits[0, -1, :].float()
            
            if do_sample and temperature > 0:
                # Apply temperature and sample
                scaled_logits = logits / temperature
                probs = F.softmax(scaled_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1).item()
            else:
                # Greedy - temperature has NO effect!
                next_token = logits.argmax().item()
            
            generated_tokens.append(next_token)
            input_ids = torch.cat([input_ids, torch.tensor([[next_token]]).to(model.device)], dim=1)
            
            if next_token == tokenizer.eos_token_id:
                break
    
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text, generated_tokens


def generate_greedy(model, tokenizer, prompt, max_new_tokens=512):
    """
    Generate text with greedy decoding (deterministic).
    Temperature has NO effect on greedy decoding!
    """
    return generate_with_temperature(
        model, tokenizer, prompt, 
        temperature=1.0, max_new_tokens=max_new_tokens, 
        do_sample=False
    )


# =============================================================================
# Evaluation Metrics
# =============================================================================

def evaluate_temperature_effect(clean_logits, noisy_logits, temperatures):
    """
    Evaluate how different temperatures affect the noisy distribution.
    
    For each temperature, computes:
    - KL divergence from clean distribution
    - JS divergence (symmetric)
    - Probability of correct (clean's argmax) token
    - Cross entropy
    
    Args:
        clean_logits: Logits from clean model
        noisy_logits: Logits from noisy model  
        temperatures: List of temperatures to test
    
    Returns:
        dict mapping temperature to metrics
    """
    clean_probs = F.softmax(clean_logits, dim=-1)
    correct_token = clean_logits.argmax().item()
    
    results = {}
    
    for temp in temperatures:
        noisy_probs = F.softmax(noisy_logits / temp, dim=-1)
        
        # KL divergence: KL(clean || noisy)
        kl_div = F.kl_div(noisy_probs.log(), clean_probs, reduction='sum').item()
        
        # JS divergence (symmetric)
        m_probs = 0.5 * (clean_probs + noisy_probs)
        js_div = 0.5 * F.kl_div(m_probs.log(), clean_probs, reduction='sum').item() + \
                 0.5 * F.kl_div(m_probs.log(), noisy_probs, reduction='sum').item()
        
        # Probability of correct token
        prob_correct = noisy_probs[correct_token].item()
        
        # Cross entropy
        cross_entropy = -(clean_probs * noisy_probs.log()).sum().item()
        
        results[temp] = {
            'kl_div': kl_div,
            'js_div': js_div,
            'prob_correct': prob_correct,
            'cross_entropy': cross_entropy,
        }
    
    return results


# =============================================================================
# GSM8K Helpers
# =============================================================================

def extract_answer(text):
    """Extract numerical answer from model output."""
    # Look for #### pattern (GSM8K format)
    match = re.search(r'####\s*([\d,\.\-]+)', text)
    if match:
        return match.group(1).replace(',', '')
    
    # Look for 'answer is X' pattern
    match = re.search(r'answer is[:\s]*([\d,\.\-]+)', text.lower())
    if match:
        return match.group(1).replace(',', '')
    
    # Look for boxed answer (common in reasoning models)
    match = re.search(r'\\boxed\{([\d,\.\-]+)\}', text)
    if match:
        return match.group(1).replace(',', '')
    
    # Look for last number in text
    numbers = re.findall(r'[\d,]+\.?\d*', text)
    if numbers:
        return numbers[-1].replace(',', '')
    
    return None


def extract_ground_truth(answer_text):
    """Extract ground truth from GSM8K answer format."""
    match = re.search(r'####\s*([\d,\.\-]+)', answer_text)
    if match:
        return match.group(1).replace(',', '')
    return None


def check_answer(pred, truth):
    """Check if predicted answer matches ground truth."""
    if pred is None or truth is None:
        return False
    try:
        return abs(float(pred) - float(truth)) < 0.01
    except:
        return pred.strip() == truth.strip()


def format_gsm8k_prompt(question, tokenizer, use_chat_template=True):
    """Format GSM8K question as prompt."""
    if use_chat_template and hasattr(tokenizer, 'apply_chat_template'):
        messages = [
            {"role": "user", "content": f"Solve this math problem step by step:\n{question}"}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        return f"Question: {question}\nLet me solve this step by step:\n"


# =============================================================================
# Reproducibility
# =============================================================================

def set_seed(seed):
    """Set random seed for reproducibility."""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# =============================================================================
# Sanity Checks
# =============================================================================

def verify_noise_injection(model, tokenizer, noise_ctx_fn, noise_scale, prompt="Hello"):
    """
    Verify that noise injection is actually working.
    
    Returns True if noise is being applied (outputs differ).
    """
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    # Clean output
    with torch.no_grad():
        clean_out = model(inputs.input_ids).logits[0, -1, :]
    
    # Noisy outputs (should differ each time)
    noisy_outs = []
    for _ in range(3):
        with noise_ctx_fn(model, noise_scale) as (noisy_model, _):
            with torch.no_grad():
                noisy_out = noisy_model(inputs.input_ids).logits[0, -1, :]
                noisy_outs.append(noisy_out.clone())
    
    # Check that noisy differs from clean
    diff_from_clean = (noisy_outs[0] - clean_out).abs().mean().item()
    
    # Check that noisy outputs differ from each other (stochastic)
    diff_between_noisy = (noisy_outs[0] - noisy_outs[1]).abs().mean().item()
    
    print(f"Mean diff from clean: {diff_from_clean:.4f}")
    print(f"Mean diff between noisy runs: {diff_between_noisy:.4f}")
    
    is_working = diff_from_clean > 0.01 and diff_between_noisy > 0.01
    print(f"Noise injection working: {is_working}")
    
    return is_working


In [ ]:
# Load model
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # Change as needed
SEED = 42
set_seed(SEED)

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()
print(f"Model loaded on {model.device}")

In [ ]:
# Test prompts
TEST_PROMPTS = [
    "The capital of France is",
    "In mathematics, the number pi is approximately",
]

## 6a: α vs Token Position (Teacher Forcing)

Measure α at each position during autoregressive generation.

In [ ]:
def measure_alpha_per_position(model, tokenizer, prompt, noise_ctx_fn, noise_scale, 
                                max_tokens=100, num_samples=3):
    """
    Measure α at each token position using teacher forcing.
    """
    # First generate clean sequence
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_ids = inputs.input_ids.clone()
    
    clean_logits_seq = []
    clean_tokens = []
    
    with torch.no_grad():
        for _ in range(max_tokens):
            outputs = model(input_ids)
            logits = outputs.logits[0, -1, :].float()
            clean_logits_seq.append(logits.cpu())
            token = logits.argmax().item()
            clean_tokens.append(token)
            input_ids = torch.cat([input_ids, torch.tensor([[token]]).to(model.device)], dim=1)
    
    # Now measure α at each position with noise
    alpha_per_pos = [[] for _ in range(max_tokens)]
    
    for _ in range(num_samples):
        with noise_ctx_fn(model, noise_scale) as (noisy_model, _):
            input_ids = inputs.input_ids.clone()
            
            for t in range(max_tokens):
                with torch.no_grad():
                    outputs = noisy_model(input_ids)
                    noisy_logits = outputs.logits[0, -1, :].float().cpu()
                
                # Compute α
                result = compute_alpha(clean_logits_seq[t], noisy_logits)
                alpha_per_pos[t].append(result['alpha'])
                
                # Teacher forcing: use clean token
                input_ids = torch.cat([input_ids, torch.tensor([[clean_tokens[t]]]).to(model.device)], dim=1)
    
    return [np.mean(alphas) for alphas in alpha_per_pos]

In [ ]:
# Measure α per position for both noise types
noise_scale = 0.05
max_tokens = 150

print(f"Measuring α per position (noise={noise_scale}, tokens={max_tokens})...")

prompt = TEST_PROMPTS[0]
print(f"\nPrompt: {prompt}")

print("\nActivation noise...")
alpha_activation = measure_alpha_per_position(
    model, tokenizer, prompt, activation_noise_context, noise_scale, max_tokens
)

print("Weight noise...")
alpha_weight = measure_alpha_per_position(
    model, tokenizer, prompt, weight_noise_context, noise_scale, max_tokens
)

print("Done!")

In [ ]:
# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

positions = range(len(alpha_activation))

# Left: Raw values
ax1 = axes[0]
ax1.plot(positions, alpha_activation, 'r-', alpha=0.7, linewidth=1.5, label='Activation')
ax1.plot(positions, alpha_weight, 'b-', alpha=0.7, linewidth=1.5, label='Weight')
ax1.set_xlabel('Token Position')
ax1.set_ylabel('α')
ax1.set_title('α vs Token Position')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Smoothed
window = 10
act_smooth = np.convolve(alpha_activation, np.ones(window)/window, mode='valid')
weight_smooth = np.convolve(alpha_weight, np.ones(window)/window, mode='valid')

ax2 = axes[1]
ax2.plot(range(len(act_smooth)), act_smooth, 'r-', linewidth=2, label='Activation (smoothed)')
ax2.plot(range(len(weight_smooth)), weight_smooth, 'b-', linewidth=2, label='Weight (smoothed)')
ax2.set_xlabel('Token Position')
ax2.set_ylabel('α (smoothed)')
ax2.set_title(f'Smoothed α (window={window})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('alpha_vs_position.png', dpi=150, bbox_inches='tight')
plt.show()

# Stats
print("\n" + "="*60)
print("STATISTICS")
print("="*60)
print(f"\nActivation noise:")
print(f"  First 10: mean α = {np.mean(alpha_activation[:10]):.4f}")
print(f"  Last 10:  mean α = {np.mean(alpha_activation[-10:]):.4f}")
print(f"  Ratio: {np.mean(alpha_activation[:10])/np.mean(alpha_activation[-10:]):.1f}x decrease")

print(f"\nWeight noise:")
print(f"  First 10: mean α = {np.mean(alpha_weight[:10]):.4f}")
print(f"  Last 10:  mean α = {np.mean(alpha_weight[-10:]):.4f}")
print(f"  Ratio: {np.mean(alpha_weight[:10])/np.mean(alpha_weight[-10:]):.1f}x decrease")

## 6b: α vs Context Length (Fixed Context)

Control for attention dilution by using fixed context length.

In [ ]:
def measure_alpha_fixed_context(model, tokenizer, prompt, noise_ctx_fn, noise_scale,
                                 context_lengths=[10, 50, 100, 150], num_samples=5):
    """
    Measure α with fixed context length (controls for attention dilution).
    """
    # First generate clean sequence
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_ids = inputs.input_ids.clone()
    
    max_len = max(context_lengths) + 10
    clean_tokens = []
    
    with torch.no_grad():
        for _ in range(max_len):
            outputs = model(input_ids)
            token = outputs.logits[0, -1, :].argmax().item()
            clean_tokens.append(token)
            input_ids = torch.cat([input_ids, torch.tensor([[token]]).to(model.device)], dim=1)
    
    results = []
    
    for ctx_len in context_lengths:
        # Build context
        context_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)
        for t in range(ctx_len):
            context_ids = torch.cat([context_ids, torch.tensor([[clean_tokens[t]]]).to(model.device)], dim=1)
        
        # Get clean logits
        with torch.no_grad():
            clean_logits = model(context_ids).logits[0, -1, :].float().cpu()
        
        # Get noisy logits
        alphas = []
        for _ in range(num_samples):
            with noise_ctx_fn(model, noise_scale) as (noisy_model, _):
                with torch.no_grad():
                    noisy_logits = noisy_model(context_ids).logits[0, -1, :].float().cpu()
                result = compute_alpha(clean_logits, noisy_logits)
                alphas.append(result['alpha'])
        
        results.append({
            'context_length': ctx_len,
            'alpha_mean': np.mean(alphas),
            'alpha_std': np.std(alphas),
        })
        print(f"  Context {ctx_len}: α = {np.mean(alphas):.4f} ± {np.std(alphas):.4f}")
    
    return results

In [ ]:
# Test fixed context
context_lengths = [5, 10, 20, 50, 100, 150]

print("Measuring α with fixed context lengths...")
print("This isolates LayerNorm effect from attention dilution.\n")

fixed_context_results = measure_alpha_fixed_context(
    model, tokenizer, TEST_PROMPTS[0],
    activation_noise_context, noise_scale,
    context_lengths=context_lengths
)

In [ ]:
# Compare teacher forcing vs fixed context
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Teacher forcing (growing context)
ax1 = axes[0]
ax1.plot(range(len(alpha_activation)), alpha_activation, 'r-', alpha=0.7, linewidth=1.5)
ax1.set_xlabel('Token Position')
ax1.set_ylabel('α')
ax1.set_title('Teacher Forcing (growing context)')
ax1.grid(True, alpha=0.3)

# Right: Fixed context
ax2 = axes[1]
ctx_lens = [r['context_length'] for r in fixed_context_results]
alphas = [r['alpha_mean'] for r in fixed_context_results]
stds = [r['alpha_std'] for r in fixed_context_results]
ax2.errorbar(ctx_lens, alphas, yerr=stds, fmt='bo-', linewidth=2, markersize=8, capsize=5)
ax2.set_xlabel('Context Length')
ax2.set_ylabel('α')
ax2.set_title('Fixed Context (single prediction)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('teacher_forcing_vs_fixed_context.png', dpi=150, bbox_inches='tight')
plt.show()

# Analysis
tf_ratio = np.mean(alpha_activation[:10]) / np.mean(alpha_activation[-10:])
fc_ratio = fixed_context_results[0]['alpha_mean'] / fixed_context_results[-1]['alpha_mean']

print("\n" + "="*60)
print("ANALYSIS")
print("="*60)
print(f"Teacher forcing α decrease: {tf_ratio:.1f}x")
print(f"Fixed context α decrease:   {fc_ratio:.1f}x")
print(f"\nConclusion: ", end="")
if fc_ratio > 1.5:
    print("Attention dilution contributes to α decrease")
else:
    print("LayerNorm is the main cause of α decrease")

## 6c: Dynamic T*(t) vs Constant T*

Compare position-dependent temperature with constant temperature.

In [ ]:
# Compute T* per position
t_star_per_pos = [np.sqrt(1 + a) for a in alpha_activation]

# Plot T* vs position
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(positions, t_star_per_pos, 'b-', linewidth=2, label='T*(t) = √(1+α(t))')
ax.axhline(y=np.sqrt(1 + np.mean(alpha_activation)), color='red', linestyle='--', 
           label=f'Constant T* = {np.sqrt(1 + np.mean(alpha_activation)):.3f}')
ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5, label='T = 1.0')

ax.set_xlabel('Token Position')
ax.set_ylabel('T*')
ax.set_title('Optimal Temperature vs Position')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('t_star_vs_position.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nT* range: {min(t_star_per_pos):.3f} to {max(t_star_per_pos):.3f}")
print(f"Mean T*: {np.mean(t_star_per_pos):.3f}")

## Summary

Key findings from this experiment:

1. **α decreases with position** - up to 15x from first to last tokens
2. **Both LayerNorm and attention dilution** contribute to this decrease
3. **Dynamic T*(t)** should outperform constant T*
4. **For long sequences**, T ≈ 1 is reasonable since α converges to low values